In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE104948'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Glomerular Transcriptome from European Renal cDNA Bank subjects and living donors"
!Series_summary	"summary : Glomerular Transcriptome from European Renal cDNA Bank subjects and living donors. Samples included in this analysis have been previously analyzed using older CDF definitions and are included under previous GEO submissions - GSE47183 (chronic kidney disease samples), and GSE32591 (IgA nephropathy samples).  "
!Series_overall_design	"RNA from the glomerular compartment of was extracted and processed for hybridization on Affymetrix microarrays, annotated using Human Entrez Gene ID custom CDF version 19."
Sample Characteristics Dictionary:
{0: ['tissue: Glomeruli from kidney biopsy'], 1: ['diagnosis: Diabetic Nephropathy', 'diagnosis: Focal Segmental Glomerular Sclerosis/Minimal Change Disease', 'diagnosis: Focal Segmental Glomerular Sclerosis', nan, 'diagnosis: Minimal Change Disease', 'diagnosis: ANCA Associated Vasculitis', 'diagnosis: Tum

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

from numpy import nan

is_gene_available = True  # Based on the background, it appears to be gene expression data.

# Identify variable keys
for key, values in {
    0: ['tissue: Glomeruli from kidney biopsy'],
    1: ['diagnosis: Diabetic Nephropathy', 'diagnosis: Minimal Change Disease', 'diagnosis: Thin Membrande Disease', 'diagnosis: Hypertensive Nephropathy', 'diagnosis: Tumor Nephrectomy', 'diagnosis: IgA Nephropathy', 'diagnosis: Focal Segmental Glomerular Sclerosis', nan, 'diagnosis: Membranous Glomerulonephropathy', 'diagnosis: Systemic Lupus Erythematosus']
}.items():
    if any(isinstance(v, str) and 'diagnosis:' in v for v in values):
        trait_row = key
    if any(isinstance(v, str) and 'age:' in v for v in values):
        age_row = key
    if any(isinstance(v, str) and 'gender:' in v for v in values):
        gender_row = key

def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    val = value.split(':')[1].strip().lower()
    if val in ['diabetic nephropathy', 'minimal change disease', 'thin membrane disease', 'hypertensive nephropathy', 'tumor nephrectomy', 'iga nephropathy', 'focal segmental glomerular sclerosis', 'membranous glomerulonephropathy', 'systemic lupus erythematosus']:
        return 1  # Presence of chronic kidney disease
    return 0  # Unknown or not chronic kidney disease

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    try:
        return float(value.split(':')[1].strip())
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    val = value.split(':')[1].strip().lower()
    return 1 if val == 'male' else 0 if val == 'female' else None

save_cohort_info('GSE104948', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE104948.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2810770': [1], 'GSM2810771': [1], 'GSM2810772': [1], 'GSM2810773': [1], 'GSM2810774': [1], 'GSM2810775': [1], 'GSM2810776': [1], 'GSM2810777': [0], 'GSM2810778': [0], 'GSM2810779': [0], 'GSM2810780': [0], 'GSM2810781': [0], 'GSM2810782': [0], 'GSM2810783': [1], 'GSM2810784': [1], 'GSM2810785': [1], 'GSM2810786': [1], 'GSM2810787': [1], 'GSM2810788': [1], 'GSM2810789': [1], 'GSM2810790': [1], 'GSM2810791': [1], 'GSM2810792': [1], 'GSM2810793': [None], 'GSM2810794': [None], 'GSM2810795': [None], 'GSM2810796': [None], 'GSM2810797': [None], 'GSM2810798': [None], 'GSM2810799': [None], 'GSM2810800': [None], 'GSM2810801': [None], 'GSM2810802': [None], 'GSM2810803': [None], 'GSM2810804': [None], 'GSM2810805': [None], 'GSM2810806': [None], 'GSM2810807': [None], 'GSM2810808': [None], 'GSM2810809': [None], 'GSM2810810': [None], 'GSM2810811': [1], 'GSM2810812': [1], 'GSM2810813': [1], 'GSM2810814': [1], 'GSM2810815': [1], 'GSM2810816': [0], 'GSM2810817': [0], 'GSM2810818': [0], 'GSM2810819': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['10000_at', '10001_at', '10002_at', '10003_at', '100048912_at',
       '10004_at', '10005_at', '10006_at', '10007_at', '100093698_at',
       '10009_at', '1000_at', '10010_at', '100126791_at', '100128124_at',
       '100128640_at', '100129128_at', '100129250_at', '100129271_at',
       '100129361_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1000_at', '10000_at', '100009676_at', '10001_at', '10004_at'], 'Symbol': ['CDH2', 'AKT3', 'ZBTB11-AS1', 'MED6', 'NAALADL1'], 'SPOT_ID': ['cadherin 2', 'AKT serine/threonine kinase 3', 'ZBTB11 antisense RNA 1', 'mediator complex subunit 6', 'N-acetylated alpha-linked acidic dipeptidase-like 1'], 'ENTREZ_GENE_ID': ['1000', '10000', '100009676', '10001', '10004']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping)
